In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
from sklearn.neural_network import MLPClassifier
# from nltk.tokenize import sent_tokenize
# from rouge import Rouge
# from sklearn.model_selection import train_test_split
# from sentence_transformers import SentenceTransformer
from sklearn.metrics import log_loss, roc_auc_score, average_precision_score
# PrecisionRecallDisplay, RocCurveDisplay
from malnis import show
import pickle

In [2]:
data_folder = "/home/jarobyte/scratch/malnis_dataset/data/"

In [3]:
train = pd.read_pickle(data_folder + "data_train.pkl")
show(train)

(7172, 9)


,query,document,summary,r1,r2,rl,sentences,relevance,original_sentences
7593,Due to the lack of structured knowledge applie...,"Index Terms—Machine learning, knowledge discov...","[Text classification [226], text clustering [2...",0.278846,0.095941,0.259615,"[Index Terms—Machine learning, knowledge disco...","[False, False, False, False, False, False, Fal...",685
7554,Obtaining enough labeled data to robustly trai...,One of the greatest roadblocks to using modern...,"[“Lawyer”); moreover, these sources might be c...",0.295455,0.104247,0.272727,[One of the greatest roadblocks to using moder...,"[True, False, True, False, False, False, False...",246
4892,"Word embeddings, i.e., low-dimensional vector ...",Recently some studies have shown that text cla...,"[A recent work (Schuster et al., 2020) showed ...",0.407018,0.186364,0.385965,[Recently some studies have shown that text cl...,"[False, False, True, False, False, False, Fals...",202
4169,"In this paper our objectives are, first, netwo...",It is commonsense that how you look at an obje...,"[Views are commonly different sensory signals,...",0.257143,0.090909,0.250000,[It is commonsense that how you look at an obj...,"[False, False, False, False, False, False, Fal...",240
6690,"This paper describes TextTiling, an algorithm ...",Accurately representing the distance between t...,[Most similar to our method is that of Wan (20...,0.206349,0.037975,0.158730,[Accurately representing the distance between ...,"[False, False, False, False, False, False, Fal...",246


In [4]:
dev = pd.read_pickle(data_folder + "data_dev.pkl")
show(dev)

(896, 9)


,query,document,summary,r1,r2,rl,sentences,relevance,original_sentences
5367,We propose the task of free-form and open-ende...,"Index Terms—Deep learning, visual analytics, i...",[Research combining both image and text data h...,0.249240,0.117400,0.218845,"[Index Terms—Deep learning, visual analytics, ...","[False, False, False, False, True, False, Fals...",497
4278,"In this paper, we propose a novel neural netwo...",Document summarization aims to produce fluent ...,"[For each sentence, we apply an RNN with Gated...",0.281407,0.112281,0.281407,[Document summarization aims to produce fluent...,"[False, False, False, False, False, False, Fal...",238
307,Learning to rank with biased click data is a w...,CCS CONCEPTS • Information systems→ Learning t...,"[Most recently, a new approach for de-biasing ...",0.321716,0.160558,0.289544,[CCS CONCEPTS • Information systems→ Learning ...,"[True, False, False, False, False, False, Fals...",401
4413,"We present the Compressive Transformer, an att...","Transformers (Vaswani et al., 2017) have achie...",[on this task as their primary evaluation (Dai...,0.254144,0.075314,0.232044,"[Transformers (Vaswani et al., 2017) have achi...","[False, False, False, False, False, False, Fal...",357
2963,We propose a method for online news stream clu...,ACM Reference Format: Doug Beeferman and Hang ...,[[17] is the first to include BERT contextual ...,0.255102,0.094203,0.255102,[ACM Reference Format: Doug Beeferman and Hang...,"[False, False, False, False, False, False, Fal...",185


In [5]:
train_targets = np.concatenate(train.relevance.to_list())
train_targets.shape

(2089747,)

In [6]:
output_folder = "/home/jarobyte/scratch/malnis_dataset/data/classical_models/"

# Words

In [7]:
%%time
# corpus = sum([d["sentences"] for d in tqdm(train_records)], start = [])
corpus = train.sentences.sum()
len(corpus)

CPU times: user 2min 21s, sys: 5.97 s, total: 2min 27s
Wall time: 2min 28s


2089747

In [8]:
%%time
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(corpus)
train_features.shape

CPU times: user 36.3 s, sys: 489 ms, total: 36.8 s
Wall time: 36.9 s


(2089747, 60166)

In [52]:
%%time 

n = 10000
X = train_features[:n]
Y = train_targets[:n]

clf = MLPClassifier(
    verbose = True, 
#     max_iter = 1
)
clf.partial_fit(X, Y, classes = np.unique(Y))

Iteration 1, loss = 0.47340644
CPU times: user 3.91 s, sys: 2.63 s, total: 6.55 s
Wall time: 6.59 s


MLPClassifier(verbose=True)

# predicting on dev set

In [53]:
dev_features = [vectorizer.transform(l) for l in tqdm(dev.sentences)]

  0%|          | 0/896 [00:00<?, ?it/s]

In [54]:
dev_preds = [clf.predict_proba(f) for f in tqdm(dev_features)]

  0%|          | 0/896 [00:00<?, ?it/s]

In [55]:
dev_targets = np.concatenate(dev.relevance.to_list())
dev_targets.shape

(264918,)

In [56]:
dev_preds_flat = np.concatenate(dev_preds)[:, 1]
dev_preds_flat.shape

(264918,)

In [57]:
average_precision_score(dev_targets, dev_preds_flat)

0.026500608650625878

In [58]:
roc_auc_score(dev_targets, dev_preds_flat)

0.4177517408660397

# saving to disk

In [46]:
with open("/home/jarobyte/scratch/malnis_dataset/words/predictions/test.pkl", "wb") as file:
    pickle.dump(clf, file)

In [47]:
with open("/home/jarobyte/scratch/malnis_dataset/words/predictions/test.pkl", "rb") as file:
    clf_2 = pickle.load(file)

In [48]:
dev_preds = [clf_2.predict_proba(f) for f in tqdm(dev_features)]

  0%|          | 0/896 [00:00<?, ?it/s]

In [49]:
dev_preds_flat = np.concatenate(dev_preds)[:, 1]
dev_preds_flat.shape

(264918,)

In [50]:
average_precision_score(dev_targets, dev_preds_flat)

0.07870122614174122

In [51]:
roc_auc_score(dev_targets, dev_preds_flat)

0.6871863337684786